In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import datasets
import matplotlib.pyplot as plt

### Prepare Data

In [3]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size = 0.2, random_state=1234)

In [4]:
#Scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [5]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

### Design Model (Input, Output, Forward Pass)

In [6]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted
    
model = LogisticRegression(n_features)

### Loss and Optimizer

In [8]:
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Training Loop

In [9]:
num_epochs = 100
for epoch in range(num_epochs):
    #forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    
    #backward pass
    loss.backward()
    
    #updates
    optimizer.step()
    
    #zero the gradient
    optimizer.zero_grad()
    
    
    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss={loss.item():.4f}')

epoch: 10, loss=0.6842
epoch: 20, loss=0.5285
epoch: 30, loss=0.4384
epoch: 40, loss=0.3805
epoch: 50, loss=0.3403
epoch: 60, loss=0.3106
epoch: 70, loss=0.2876
epoch: 80, loss=0.2693
epoch: 90, loss=0.2542
epoch: 100, loss=0.2415


In [10]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum()/float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

accuracy = 0.9035
